# 基础概念
* 分布式场景下的事务一致性：
    * 参考资料：
        * https://www.cloud.alipay.com/articles/middleware/transfer_solutions
        * http://dbaplus.cn/news-141-1175-1.html
    * 2PC：
        * 实现方式：TCC，JTA/XA
        * 事务处理流程：
            * 在第一阶段，协调者询问所有的参与者是否可以提交事务（请参与者投票），所有参与者向协调者投票
            * 在第二阶段，协调者根据所有参与者的投票结果做出是否事务可以全局提交的决定，并通知所有的参与者执行该决定，当且仅当所有的参与者都同意才提交事务，只要有一个参与者投票选择放弃事务，则事务必须被放弃
        * 优点：强一致性
        * 缺点：
            * 容错能力较差，比如在节点宕机或者超时的情况下，无法确定流程的状态，只能不断重试
            * 性能较差， 消息交互多，且受最慢节点影响，资源被锁住
    * JTA/XA解决方案：
        * 事务处理流程：
            * 分别开启A数据库、B数据库上的XA事务
            * 分别在A数据库XA事务上执行A账户扣钱任务，在B数据库XA事务中执行B账号加钱任务
            * 执行XA事务一阶段的预提交
            * 如果A数据库、B数据库上的XA事务预提交均成功，则提交XA事务
            * 如果A数据库、B数据库上的XA事务预提交出现失败，则回滚XA事务
        * 相关特性：
            * XA并发性能受限，XA事务内访问的数据都会被数据库加锁，直到XA事务提交或者回滚，这些数据锁才会被释放
            * 运维成本高，需要自己设计恢复程序，寻找为完成的任务，并执行剩余的操作
    * TCC：Try、Commit、Cancel
        * 事务处理流程：
            * 首先执行各TCC参与者的一阶段方法，做转账准备操作；一阶段准备成功，则二阶段的提交一定能成功
            * 如果所有一阶段参与者方法均执行成功，那么二阶段转账操作会去执行所有TCC参与者的提交方法
            * 如果一阶段有任一参与者出现失败，那么二阶段便会执行所有 TCC参与者的回滚方法，使得各账户恢复至转账前的状态
        * 相关特性：
            * 位于业务服务层而非资源层，由业务层保证原子性
            * 无资金安全风险，一阶段只会冻结转账资金，不会发生真正的资金转账，只有第一阶段成功后才会真正转账
            * 运维成本低，无需维护恢复程序
            * 性能优越，无任何数据库层面的全局数据锁
    * 基于消息的分布式事务
        * 事务处理流程：
            * 转账服务内部使用消息事务功能，发送加钱消息给加钱服务，同时调用扣钱服务完成A账户的扣钱
            * 调用加钱服务完成B账户的加钱，如果消息消费失败，消息队列下个周期重试投递消息
        * 相关特性：
            * 消息消费延迟，一阶段A账户扣款之后，转账操作便结束，此时用户认为转账操作已经完成，但实际到账时间不确定
            * 要求二阶段的消息消费必须成功，因为此时A账户已经扣完款，无法撤销
            * 引入消息队列风险点，消息事务可用的前提是消息队列可用，消息队列出现消息积压会导致二阶段延迟更加严重
    * 冲正补偿解决方案
        * 事务处理流程：
            * 先执行A账户的正向操作A账户扣钱操作，如果执行失败，则执行逆向操作A账户扣钱回滚操作，最终转账操作失败
            * 如果A账户正向操作成功，则执行B账户的正向操作B账户加钱操作；如果B账户加钱执行失败，则会执行B账户逆向操作B账户加钱回滚操作，以及A账户的逆向操作A账户扣钱回滚操作，最终转账失败
        * 相关特性：
            * 接入成本高，冲正补偿需要用户设计实现各服务的正向和逆向操作，用户在设计正向操作时，需要同时考虑逆向操作该如何执行；需要在正向操作中保存一些中间数据，供逆向操作运行时使用，系统设计实现较复杂
            * 资金安全问题，因为直接操作的是用户的账户，当钱已经到B账户，并且被B转走后，此时触发的回滚操作都会导致数据不一致
            * 维护成本高，需要自己设计恢复程序，寻找为完成的任务，并执行剩余的操作

****

# 接口设计
* 接口规范：
    * 参考资料：https://docs.open.alipay.com/309/106237/
    * request body：使用服务端的public key加密传输
        * applicationId：服务端为客户端生成的应用id
        * transformNumber：转账交易号，每次转账生成一个唯一的编号
        * transformerName：转账人姓名
        * transformeeAccount：被转账人账户
        * transformeeName：被转账人姓名
        * transformMoney：转账资金
        * signString：服务端授予的签名
* 安全，资金安全：
    * 签名认证过程：
        * 客户端先生成自己的公钥私钥对，将公钥发送到服务端
        * 服务端生成客户端的唯一应用ID，并用自己的私钥进行签名
        * 服务端通过客户端发过来的公钥，对应用ID明文，签名加密字符串，服务端的公钥加密并发送给客户端
        * 客户端通过自己的私钥解密，并用服务端的公钥认证签名字符串
        * 本接口只涉及转账核心功能，因此该签名过程省略，使用提前准备好的服务端公钥密钥，客户端公钥密钥完成安全认证
    * 使用RSA对称加密，完成身份认证
        * 客户端使用服务端的公钥对传输消息加密
        * 服务端使用私钥解密消息，并使用自己的公钥验证签名字符串与应用ID是否匹配
        * 只有认证通过才可以进行后续的转账工作
    * 使用RSA对称加密，完成消息传输
        * 客户端使用服务端的公钥对传输消息加密
        * 服务端使用私钥解密消息，当身份认证通过后，处理转账过程
        * 服务端通过客户端的公钥加密处理结果，并发送到客户端（这个步骤可省略，当传输消息不涉及隐私消息时）
        * 客户端通过自己的私钥解密返回结果（这个步骤可省略，当传输消息不涉及隐私消息时）
* 幂等：
    * 通过交易号，保障接口的幂等，客户端重复提交时，通过判断交易号的处理状态选择处理方式
        * 当交易号不存在时，处理转账
        * 当交易号存在，且不是失败状态时，直接返回
        * 当交易号存在，且是失败状态时，处理转账（使用数据库中失败的交易信息处理转账，不会使用这次提交的信息，因为如果处理失败并返回给客户端后，客户端再次提交时应该生成一个新的交易号）
* 重试：
    * 服务端将重试转账过程3次，超过3次不成功才会通知客户端处理失败
* 分布式场景下的事务一致性，并发，用户投诉
    * 使用TCC分布式事务处理思想，详情参考基础概念中的介绍
* 异常分支
    * 身份认证不通过
    * 交易号已经存在，参考幂等处理
    * 账号不存在，返回失败
    * 账号不匹配，返回失败
    * 账号余额不足，返回失败
    * 系统异常，通过重试解决，重试解决不了返回失败
* 第三方框架以及工具：
    * springboot
    * springjpa，多数据源配置
    * mysql
    * intellij idea
    * mac
    * gradle
    * flyway